<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<font color="#138D75">**Copernicus EUMETSAT**</font> <br>
**Copyright:** 2025 EUMETSAT <br>
**License:** MIT <br>
**Author:** Anna-Lena Erdmann (EUMETSAT)

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://jupyterhub.prod.wekeo2.eu/hub/user-redirect/lab/tree/public/wekeo4data/wekeo-gpu/01_Introduction_to_the_WEkEO_Workspace_GPUs.ipynb"><img src="https://img.shields.io/badge/launch-WEKEO-1a4696.svg?style=flat&logo=" alt="Open in WEkEO"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>    
</html>

<div class="alert alert-block alert-success">
<h3> Getting Started with the WEkEO Workspace GPUs </h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
  - **<a href="https://my.wekeo.eu/user-registration" target="_blank">A WEkEO account</a>**
  - **Execution of the notebook under the WEkEO Workspace <a href="https://help.wekeo.eu/en/articles/7945473-which-are-the-computing-resources-of-the-wekeo-jupyterhub" target="_blank"> Machine Learning (GPU) server</a>**
  

</div>
<hr>

# Introduction to the WEkEO Workspace GPUs

### Learning outcomes

At the end of this notebook you will know:

* how to access the WEkEO Workspace GPU Server
* how to monitor GPU usage 
* how to compare runtimes of GPUs and CPUs 


### Outline

In this notebook, you will learn how to access and use the GPU resources available in the WEkEO Workspace to accelerate machine learning tasks. It is the starting point to get familiarized with GPUs and lays the basis for upcoming Machine Learning use cases. You will use the GPU with a simple tensor multiplication, monitor GPU usage during the calculations, and compare the performance of CPU and GPU runtimes. This example provides a hands-on introduction to working with GPU-accelerated deep learning in the WEkEO Workspace. 

<div class="alert alert-info" role="alert">

### Contents <a id='totop'></a>

</div>

1. [Accessing the WEkEO Workspace GPU Environment](#section0)  
2. [Monitoring GPU Usage](#section1)   
3. [Comparing GPU and CPU Runtimes](#section2)  


<hr>

<div class="alert alert-info" role="alert">

## 1. <a id='section0'></a>Accessing the WEkEO Workspace GPU Environment
[Back to top](#totop)
    
</div>

To get started with GPU-accelerated machine learning in the WEkEO platform, follow these steps:

1. Log in to your [WEkEO user account](https://www.wekeo.eu/) and go to your **personal dashboard**.  


<p align="center">
  <img src="img/wekeo-landing-page.png" alt="Landing Page" style="width:60%;">
</p>

2. In the dashboard, navigate to the **"Workspace"** tab and launch the **WEkEO JupyterLab environment**.  


<p align="center">
  <img src="img/wekeo-personal-dashboard.png" alt="WEkEO Personal Dashboard" style="width:60%;">
</p>

3. Once prompted, select the server type. Choose **"Machine Learning (GPUs)"** to request a server with GPU resources.  


<p align="center">
  <img src="img/gpu-server.png" alt="GPU Server Selection" style="width:60%;">
</p>

4. After the environment loads, open a **Terminal** from the JupyterLab launcher.

To verify that the GPU is available, you can run the following command in the terminal:

`nvidia-smi`

<p align="center">
  <img src="img/nvidia-smi.png" alt="GPU Server Selection" style="width:60%;">
</p>

Here, you can see that **6144 MB of GPU Memory** are available in your JupyterHub environment. You can run the `nvidia-smi` command anytime you want to check if you are in the GPU server or on the non-GPU Earth Observation / Machine LEarning Server. 

<div class="alert alert-info" role="alert">

## 2. <a id='section1'></a>Monitor GPU Usage
[Back to top](#totop)
    
</div>

When working on a GPU-enabled machine, it's useful to monitor your GPU's memory usage and activity to make sure your code is running efficiently. One of the most user-friendly tools for this is `nvtop`.

### What is `nvtop`?

`nvtop` (NVIDIA TOP) is an interactive command-line utility similar to `htop`, but specifically designed for monitoring NVIDIA GPUs. It displays real-time information on:
- GPU memory usage
- Utilization percentage
- Active processes using the GPU
- Power consumption (if available)

It is ideal for checking whether your training process is actively using the GPU and how much memory is being consumed.


### Install nvtop to the environment

You can run in the terminal: 

```bash
    conda install -c conda-forge nvtop -y
```

Or execute the code cell below:


In [2]:
%conda install -c conda-forge -q nvtop -y

Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


This installs the GPU monitoring tool in your Conda environment. 

Note: this will not be persistent if you install it in the Python3 environment of the JupyterHub. Create a new custom environment to have the tool persistently in your environment. 


### Run `nvtop`

Once installed, you can monitor your GPU in real time by opening a **terminal** in JupyterLab and running:

```bash
nvtop
```

The monitoring tool will open in the terminal. You see two lines: one for the GPU memory and one for the GPU utilization. 

<p align="center">
  <img src="img/nvtop.png" alt="nvtop" style="width:60%;">
</p>

### Start a matrix calculation

To see the GPU monitoring tool in action, we will start a simple tensor multiplication in pytorch. 

In [4]:
import torch
import time
import warnings

# Suppress warnings (e.g., related to data type conversions or GPU init)
warnings.filterwarnings("ignore")
# Set the size of the matrix
matrix_size = 10000

# Generate two random matrices
a = torch.randn(matrix_size, matrix_size)
b = torch.randn(matrix_size, matrix_size)

# Move the matrices to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a_gpu = a.to(device)
b_gpu = b.to(device)


# Measure GPU time 
_ = torch.matmul(a_gpu, b_gpu)  
torch.cuda.synchronize()        
result_gpu = torch.matmul(a_gpu, b_gpu)
torch.cuda.synchronize()       




When executing the matrix multiplication, we can see the level of GPU memory and GPU utilization changing in `nvtop`:

<p align="center">
  <img src="img/matrix-multiplication.png" alt="nvtop" style="width:60%;">
</p>

Note, that even after the calculation is finished (blue line), the GPU memory is still occupied. To release the GPU memory, we can clear the GPU cache. 

In [5]:
torch.cuda.empty_cache()

<div class="alert alert-info" role="alert">

## 3. <a id='section2'></a>Comparing GPU and CPU Runtimes
[Back to top](#totop)
    
</div>

One of the key advantages of using the WEkEO Workspace GPU server is the significant speed-up it can offer for large-scale numerical computations. To demonstrate this, we'll compare how long it takes to perform a large matrix multiplication using the CPU versus the GPU.

This type of operation is common in machine learning tasks like neural network training, where many matrix operations are performed in sequence.


We will compute the product of two large square matrices (3000 × 3000) using PyTorch, once on the CPU and once on the GPU. It is very similar to the code above, except that we run the matrix multiplication on both, the GPU and the CPU. 

In [6]:
# Set the matrix size
matrix_size = 3000

# Create random matrices on the CPU
a_cpu = torch.randn(matrix_size, matrix_size)
b_cpu = torch.randn(matrix_size, matrix_size)

# Also move the matrices to the GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a_gpu = a_cpu.to(device)
b_gpu = b_cpu.to(device)

# Measure CPU computation time
start_cpu = time.time()
result_cpu = torch.matmul(a_cpu, b_cpu)
end_cpu = time.time()
cpu_time = end_cpu - start_cpu

# Warm-up and measure GPU computation time

start_gpu = time.time()
result_gpu = torch.matmul(a_gpu, b_gpu)
torch.cuda.synchronize()
end_gpu = time.time()
gpu_time = end_gpu - start_gpu

# Print results
print(f"CPU time: {cpu_time:.4f} seconds")
print(f"GPU time: {gpu_time:.4f} seconds (if GPU available)")

CPU time: 0.4448 seconds
GPU time: 0.0020 seconds (if GPU available)


You can see from the results, that the GPU is performing the martix calculation **40x faster** than the CPU!

---

## Congratulations!

You have successfully completed the first notebook about the WEkEO WOrkspace GPUs! You are now equipped with all the tools you require to get started with GPUs for you EO projects. 

Do you want to find out what use cases you can do next with the WEkEO Workspace GPUs? Ceck out the [notebook on vision-language models](https://jupyterhub.prod.wekeo2.eu/hub/user-redirect/lab/tree/public/wekeo4data/wekeo-gpu/ollama_image_description.ipynb) and how to generate satellite data descriptions with them. 